In [18]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import TensorDataset, DataLoader

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

torch.manual_seed(1)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

x = torch.rand((4096, 3)).to(device)
y_transform = lambda x: (x[:, 0]**3)*3 + 4*(x[:, 1]**2) + (4*x[:, 2])
y = y_transform(x).to(device)

class myModel(nn.Module):
    def __init__(self, input_dims):
        super().__init__()
        self.dims = 1024
        self.model = nn.Sequential(
            nn.Linear(input_dims, self.dims),
            nn.ReLU(),
            nn.Linear(self.dims, self.dims),
            nn.ReLU(),
            nn.Linear(self.dims, 1)            
        )
        
    def forward(self, x):
        x = self.model(x)
        return x

    
model = myModel(x.shape[1]).to(device)
optimizer = optim.AdamW(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

batch_size = 8
dataset = TensorDataset(x, y)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

epochs = 50

for epoch in range(epochs):
    
    for x_batch, y_batch in dataloader:
        optimizer.zero_grad()  # Zero the gradients

        y_pred = model(x_batch)      # Get predictions from the model

        loss = loss_fn(y_pred, y_batch.view(batch_size, -1))  # Compute the loss

        loss.backward()       # Backpropagate the loss

        optimizer.step()      # Update the model parameters

    if epoch % (epochs / 10) == 0:
        print(f'epoch = {epoch}, loss = {loss.item()}')
        
print('test')
print(model(torch.tensor([3.0,4.0,5.0]).to(device)))
print('real')
print(y_transform(torch.tensor([[0.7,0.3,0.1]]).to(device)))


epoch = 0, loss = 0.0020421347580850124
epoch = 5, loss = 0.0039193653501570225
epoch = 10, loss = 0.00018892926163971424
epoch = 15, loss = 0.002619548700749874
epoch = 20, loss = 0.00032010232098400593
epoch = 25, loss = 0.002933270065113902
epoch = 30, loss = 0.0009258439531549811
epoch = 35, loss = 0.0006608207477256656
epoch = 40, loss = 9.974760178010911e-05
epoch = 45, loss = 0.0006053188117220998
test
tensor([66.9830], device='cuda:0', grad_fn=<AddBackward0>)
real
tensor([1.7890], device='cuda:0')


In [19]:
print('test')
print(model(torch.tensor([0.7,0.3,0.1]).to(device)))
print('real')
print(y_transform(torch.tensor([[0.7,0.3,0.1]]).to(device)))

test
tensor([1.7842], device='cuda:0', grad_fn=<AddBackward0>)
real
tensor([1.7890], device='cuda:0')
